In [16]:
%pip install -q langchain-elasticsearch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
%pip install -q sentence-transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from uuid import uuid4
from elasticsearch import Elasticsearch
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_elasticsearch import ElasticsearchStore
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.retrievers import ElasticSearchBM25Retriever

**Загрузка базы знаний:**

In [2]:
with open(
    file=r"C:\Users\andre\AdvancedRAG\static\texts\ТИУ База знаний.txt",
    mode="r",
    encoding="utf-8"
) as file:
    text = file.read()

print(f"Длина текста: {len(text)}")

Длина текста: 71987


**Разделение текста на чанки:**

In [3]:
'''text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)'''

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=20,
    length_function=len,
)

chunks = text_splitter.create_documents([text])

N = 5

print(f"Всего чанков: {len(chunks)}")
print(f"Первые {N} чанков:")
chunks[:N]

Всего чанков: 173
Первые 5 чанков:


[Document(page_content='Основные сведения:\nПолное наименование образовательной организации\nФедеральное государственное бюджетное образовательное учреждение высшего образования «Тюменский индустриальный университет»\nНаименование организации на иностранном языке\nFederal State Budget Educational Institution of Higher Education «Industrial University of Tyumen»/ IUT\nСокращенное наименование образовательной организации (при наличии)\nФГБОУ ВО «Тюменский индустриальный университет», ФГБОУ ВО «ТИУ», Тюменский индустриальный университет, ТИУ\nДата создания образовательной организации: 08 января 1964 г.'),
 Document(page_content='Краткая информация об образовательной организации:\nТюменский индустриальный университет – опорный вуз региона, лидер в подготовке инженерных кадров.\nИнформация о месте нахождения образовательной организации:\nЮридический адрес: 625000, Уральский федеральный округ, Тюменская область, г. Тюмень, ул. Володарского, 38\nФактический адрес: 625000, Уральский федеральны

**Загрузка Embedding мoдели:**

In [5]:
embeddings = HuggingFaceEmbeddings(
    model_name="ai-forever/sbert_large_nlu_ru",
    model_kwargs={"device": "cpu"},
    encode_kwargs={'normalize_embeddings': False},
)

C:\Users\andre\AppData\Local\Temp\ipykernel_17332\527724830.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(


**Подключение в базе данных (Elastic-search):**

In [6]:
elastic_client = Elasticsearch(
    hosts="http://localhost:9200",
    basic_auth=("elastic", "password")
)

In [6]:
indices = elastic_client.cat.indices(h='index').split()

# Удаление всех индексов
for index in indices:
    print(f"Удаляю индекс: {index}")
    elastic_client.indices.delete(index=index, ignore=[400, 404])

print("Все индексы удалены.")

Удаляю индекс: tyuiu_index
Удаляю индекс: docs-tyuiu-index
Все индексы удалены.


C:\Users\andre\AppData\Local\Temp\ipykernel_4052\3432629537.py:6: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  elastic_client.indices.delete(index=index, ignore=[400, 404])


In [7]:
elastic_store = ElasticsearchStore(
    es_url="http://localhost:9200",
    index_name="tyuiu_index",
    embedding=embeddings,
    es_user="elastic",
    es_password="password",
)

**Загрузка документов:**

In [8]:
import os


def read_files_in_directory(directory_path: str) -> str:
    files = []
    if not os.path.exists(directory_path):
        return
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        files.append(file_path)
    return files


directory_path = r'C:\Users\andre\AdvancedRAG\static\chanks'
files = read_files_in_directory(directory_path)

In [9]:
def get_file_name(file_path: str) -> str:
    return file_path.split("\\")[-1]

In [11]:
elastic_store.add_documents(documents=chunks)

['f87c3107-b3e6-477c-9e0a-f7241df2e4e9',
 '251c5519-a6c7-4ad4-899b-6676c9854fee',
 '15fd9533-985c-4c47-8242-6a90f60765ab',
 'adc751e1-ebc8-49f2-91c8-353734eb2319',
 '81e5df6e-2574-49be-9685-583aaac69269',
 'd7d319dd-e6a3-422a-9ae1-6bb62c470e5f',
 '3c111dd9-688c-41d8-8130-77d6bf93cf59',
 '180d464d-3bbc-442c-9762-22f2e6f61e4c',
 '5762852b-9762-4952-a278-5d6750f50a74',
 'e7338833-48fe-497c-81c2-0b91e0c61ee6',
 'c6cba3a2-ceeb-46c5-b761-8f3a8e563568',
 '9a3ddc45-1953-4ccb-9547-2e0d5de47305',
 'b962613d-663c-41f7-88eb-d4cf611fce39',
 '1f8c19d1-3cdd-442d-a886-f0bd87d7b804',
 'e844680e-7e4e-4d63-9745-57552384fc6d',
 '798c0225-46be-411e-af84-f5a6f4328767',
 '8a7e3dc9-5020-4c2b-9787-8adb788eba4a',
 '5b9359ad-c1df-4bbb-95e3-d53d426dce18',
 '2c696857-ed02-4a5d-aa9d-0b36f47d335f',
 'eb5e7cea-28a1-4b9c-825a-7509fde0b131',
 '0323cc27-62b8-4d6e-82b8-94476e565258',
 '7e0e9f74-3ca0-4588-b59a-c548f54e790d',
 'ae972b7e-cb29-4da1-b0a7-c026968a632e',
 '875b62e3-7649-44cb-9fc6-7500911fd6c7',
 'cda5f8b9-07aa-

In [8]:
docs = elastic_store.similarity_search("какой минимальный проходной балл на направление архитектура")
docs

[Document(page_content='В области строительства и транспорта по специальностям «Автомобильные дороги», «Объекты транспортной инфраструктуры», «Производство и применение строительных материалов» и «Организация инвестиционно-строительной деятельности» проходной балл составляет 43 (платно) и 51 (бюджет).'),
 Document(page_content='Курсы ориентированы на подготовку учащихся 10-11 классов к поступлению в ВУЗы по направлениям: 07.03.01 Архитектура, 07.03.03 Дизайн архитектурной среды, 54.03.01 Дизайн.\nПреимущества обучения в Центре архитектурной подготовки:\n- высококвалифицированный педагогический состав, в том числе члены Союза архитекторов, дизайнеров и художников России;\n- программа обучения базируется на специфике архитектурного и дизайнерского творчества;\n- благодаря полученным навыкам выпускники центра поступают в профильные ВУЗы страны и начинают свою деятельность в области архитектуры и дизайна;'),
 Document(page_content='Для направлений, связанных с экономикой и управлением, нап

In [9]:
bm25_retriever = ElasticSearchBM25Retriever(
    client=elastic_client, 
    index_name="docs-tyuiu-index",
    search_kwargs={"k": 2}
)

In [14]:
bm25_retriever.add_texts([document.page_content for document in chunks])

['d27ad876-685b-496b-8d05-8861f4858333',
 'ac5bc96e-c606-4f6d-a3e7-7b9cab433f8b',
 '3acb6fd6-5574-4134-a573-9c24e1946a68',
 '67774664-cbb7-4cb6-9603-07bb1b5985a0',
 '5f9719a1-2413-40a7-8c26-0305dae56aa4',
 'dc0feb4a-ff64-46ae-ada4-7ebf932e6c0d',
 '410c2028-6d65-48c9-91ed-c03a486d5e3c',
 '62b5a305-0ad3-4354-afaf-808a47ad3e07',
 '84008b2e-057b-4328-a793-921dcf2b4356',
 '545439a6-b518-47bf-8b5e-b15c0705808a',
 '022a2dc7-a011-410f-b45e-6dd79dfb1ae7',
 '0a46d3dc-f4f0-41c6-a23e-50de897878f3',
 '758d21e4-6c2a-4375-943e-ee8013b66ee3',
 '15c98d70-b459-417a-8b1a-de9b17b4fe10',
 'f06c2692-6c68-400e-90f5-05ad011b22c2',
 '9a9763e3-06de-4c71-a80b-dcfc7fdb7439',
 '7a207b26-b619-4f9e-be3f-bcb6b42734aa',
 '0ff358cd-beb2-491c-a842-ff03cfe0dd1f',
 'cbe0884f-abb2-4189-8663-d5f9f51b0581',
 '6731a5c0-66ae-4b67-bbfb-7f2af3006d29',
 '3f8dba6d-4aae-4415-9ad0-f81f5b5aaa67',
 'd7fffac2-6a14-447d-aaa5-cf417de94360',
 'ed9ca39f-0670-41cf-bd69-9508efeb458f',
 '04a8cec7-7e12-4e05-9168-769744eb25e5',
 'f49456f2-d112-

In [10]:
vector_store_retriever = elastic_store.as_retriever(search_kwargs={"k": 2})

In [11]:
from langchain.retrievers import EnsembleRetriever

In [29]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[vector_store_retriever, bm25_retriever],
    weights=[0.6, 0.4],
)

In [1]:
config = {"configurable": {"search_kwargs_elasticsearch_store": {"k": 1}}}
docs = ensemble_retriever.invoke("Какой проходной балл на архитектуру", config=config)
print(f"Всего документов найдено: {len(docs)}")
for doc in docs:
    print(doc)

NameError: name 'ensemble_retriever' is not defined